### Imports

In [83]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

In [84]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
import os, sys
from pathlib import Path

os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
sys.path.append('/home/k64835/Master-Thesis-SITS')

scripts_path = Path("../Data-Preprocessing/").resolve()
sys.path.append(str(scripts_path))

scripts_path = Path("../Evaluation/").resolve()
sys.path.append(str(scripts_path))

In [86]:
import pickle
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestCentroid
from scripts.data_visualiser import *
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from scripts.data_loader import *
from scripts.data_preprocessor import *
from scripts.temporal_data_preprocessor import *
from scripts.temporal_data_loader import *
from scripts.temporal_visualiser import *
from scripts.temporal_chanel_refinement import *
from model_scripts.model_helper import *
from model_scripts.pretrained_temporal_feature_extraction import *
from model_scripts.dataset_creation import *
from model_scripts.train_model_ae import *
from model_scripts.model_visualiser import *
from evaluation_scripts.patch_evaluation_helper import get_clustering_accuracy
from model_scripts.clustering import *
from model_scripts.subpatch_extraction import *
from evaluation_scripts.result_visualiser import *
from evaluation_scripts.evaluation_helper import *
from Pipeline.temporal_preprocessing_pipeline import *
import numpy as np
import config
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
import skimage.measure
import torch
import torch.nn as nn
import torch.optim as optim

## Pre-trained Model 1: Resnet 3D trained on Video Data

### Dataset prep: RGB

Data: Extracted and Pre-processed Patches (each patch containing a sugarbeet field)

Dimensions: (N, T, C, H, W) = (N, 7, 10, 64, 64)

In [16]:
preprocessing_pipeline = PreProcessingPipelineTemporal()
field_numbers_train, acquisition_dates_train, patch_tensor_train, images_visualisation_train = preprocessing_pipeline.get_processed_temporal_cubes('train', 'rgb')
field_numbers_eval, acquisition_dates_eval, patch_tensor_eval, images_visualisation_eval = preprocessing_pipeline.get_processed_temporal_cubes('eval', 'rgb')
patch_tensor_train.shape, patch_tensor_eval.shape

(torch.Size([2425, 7, 3, 64, 64]), torch.Size([48, 7, 3, 64, 64]))

Resize to match the expected spatial dimensions as required by the model: 112 x 112

In [18]:
resized_tensor_train = resize_images_transfer_learning(patch_tensor_train, (112, 112))
resized_tensor_eval = resize_images_transfer_learning(patch_tensor_eval, (112, 112))
resized_tensor_train.shape, resized_tensor_eval.shape

(torch.Size([2425, 7, 3, 112, 112]), torch.Size([48, 7, 3, 112, 112]))

Create Patch-level Data Loaders

In [19]:
train_patches_dl, test_patches, train_field_numbers, test_field_numbers = train_test_split(
    resized_tensor_train, field_numbers_train, test_size=1-0.75, random_state=10
)

dataloader_train = create_data_loader(train_patches_dl, train_field_numbers, batch_size=config.ae_batch_size, shuffle=True)
dataloader_test = create_data_loader(test_patches, test_field_numbers, batch_size=config.ae_batch_size, shuffle=False)
dataloader_eval = create_data_loader(resized_tensor_eval, field_numbers_eval, batch_size=config.ae_batch_size, shuffle=False)

### Extract Features

In [20]:
device = 'cuda'
resnet3D_extractor = ResNet3DFeatureExtractor()

tf_features_train, train_coord_dl = extract_features(resnet3D_extractor, dataloader_train, device)
tf_features_test, test_coord_dl = extract_features(resnet3D_extractor, dataloader_test, device)
tf_features_eval, eval_coord_dl = extract_features(resnet3D_extractor, dataloader_eval, device)

print("ResNet3D Extracted Features size:", tf_features_train.shape[1])  

ResNet3D Extracted Features size: 512


### K-means

In [21]:
kmeans = kmeans_function(tf_features_train.cpu(), n_clusters=2, random_state=11)

train_patch_predictions = kmeans.predict(tf_features_train.reshape(tf_features_train.size(0), -1).numpy().astype(np.float32))
test_patch_predictions = kmeans.predict(tf_features_test.reshape(tf_features_test.size(0), -1).numpy().astype(np.float32))
eval_patch_predictions = kmeans.predict(tf_features_eval.reshape(tf_features_eval.size(0), -1).numpy().astype(np.float32))

### Evaluation

Ensure all predictions are not zeros/ones

In [22]:
np.unique(eval_patch_predictions)

array([0, 1], dtype=int32)

Clustering Accuracy: Convert sub-patch level labels to patch-level labels and compare with ground truth

In [33]:
disease, acc, precision, recall, f1_score, f2_score = evaluate_clustering_metrics_patch_level(field_numbers_eval, eval_patch_predictions, config.labels_path)
print("Disease cluster:", disease)
print("Accuracy:",acc)
print("Precision:",precision)
print("Recall:",recall)
print("F1-score:",f1_score)
print("F2-score:", f2_score)

Disease cluster: 0
Accuracy: 49.18
Precision: 54.35
Recall: 71.43
F1-score: 61.73
F2-score: 67.2


Save Model

In [34]:
# with open(config.resnet3D_path, 'wb') as file:
#     pickle.dump(kmeans, file)

## Pre-trained Model 2: ViT pre-trained on ImageNet

### Use patch level RGB images

In [43]:
preprocessing_pipeline = PreProcessingPipelineTemporal()
field_numbers_train, acquisition_dates_train, patch_tensor_train, visualisation_train = preprocessing_pipeline.get_processed_temporal_cubes('train', 'rgb')
field_numbers_eval, acquisition_dates_eval, patch_tensor_eval, visualisation_eval = preprocessing_pipeline.get_processed_temporal_cubes('eval', 'rgb')
patch_tensor_train.shape, patch_tensor_eval.shape

(torch.Size([2425, 7, 3, 64, 64]), torch.Size([48, 7, 3, 64, 64]))

In [44]:
resized_tensor_train = resize_images_transfer_learning(patch_tensor_train, (224, 224))
resized_tensor_eval = resize_images_transfer_learning(patch_tensor_eval, (224, 224))
resized_tensor_train.shape, resized_tensor_eval.shape

(torch.Size([2425, 7, 3, 224, 224]), torch.Size([48, 7, 3, 224, 224]))

Create Patch-level Data Loaders

In [45]:
dataloader_train = create_data_loader(resized_tensor_train, field_numbers_train, batch_size=config.batch_size, shuffle=True)
dataloader_eval = create_data_loader(resized_tensor_eval, field_numbers_eval, batch_size=config.batch_size, shuffle=False)

### Extract Features

In [46]:
device = 'cuda'
vit_extractor = VisionTransformerExtractor()

tf_features_train, train_coord_dl = extract_features(vit_extractor, dataloader_train, device)
tf_features_eval, eval_coord_dl = extract_features(vit_extractor, dataloader_eval, device)

print("Vision Transformer (trained on imagenet) Extracted Features Shape:", tf_features_train.shape[1])  

Vision Transformer (trained on imagenet) Extracted Features Shape: 768


### K-means

In [47]:
kmeans = kmeans_function(tf_features_train.cpu(), n_clusters=2, random_state=12)

train_patch_predictions = kmeans.predict(tf_features_train.reshape(tf_features_train.size(0), -1).numpy().astype(np.float32))
eval_patch_predictions = kmeans.predict(tf_features_eval.reshape(tf_features_eval.size(0), -1).numpy().astype(np.float32))

### Evaluation

Ensure all predictions are not zeros/ones

In [48]:
np.unique(eval_patch_predictions)

array([0, 1], dtype=int32)

Clustering Accuracy: Convert sub-patch level labels to patch-level labels and compare with ground truth

In [49]:
disease, acc, precision, recall, f1_score, f2_score = evaluate_clustering_metrics_patch_level(field_numbers_eval, eval_patch_predictions, config.labels_path)
print("Disease cluster:", disease)
print("Accuracy:",acc)
print("Precision:",precision)
print("Recall:",recall)
print("F1-score:",f1_score)
print("F2-score:", f2_score)

Disease cluster: 0
Accuracy: 50.82
Precision: 56.1
Recall: 65.71
F1-score: 60.53
F2-score: 63.54


Save Model

In [51]:
# with open(config.vit_imagenet_path, 'wb') as file:
#     pickle.dump(kmeans, file)

## Pre-trained Model 3: ViT pre-trained on Sentinel-2

### Prepare the dataset
Use patch level images

In [52]:
preprocessing_pipeline = PreProcessingPipelineTemporal()
field_numbers_train, acquisition_dates_train, patch_tensor_train, visualisation_train = preprocessing_pipeline.get_processed_temporal_cubes('train', 'b10')
field_numbers_eval, acquisition_dates_eval, patch_tensor_eval, visualisation_eval = preprocessing_pipeline.get_processed_temporal_cubes('eval', 'b10')
patch_tensor_train.shape, patch_tensor_eval.shape

(torch.Size([2425, 7, 10, 64, 64]), torch.Size([48, 7, 10, 64, 64]))

Resize to 224 x 224 so that it fits the model

In [60]:
resized_tensor_train = resize_images_multichannel(patch_tensor_train, (224, 224))
resized_tensor_eval = resize_images_multichannel(patch_tensor_eval, (224, 224))
resized_tensor_train.shape, resized_tensor_eval.shape

(torch.Size([2425, 7, 10, 224, 224]), torch.Size([48, 7, 10, 224, 224]))

Get Last Timestep images 

In [87]:
train_last_timestep_patches = get_last_timestep_patches(resized_tensor_train)
test_last_timestep_patches = get_last_timestep_patches(resized_tensor_eval)

test_last_timestep_patches.shape

torch.Size([48, 10, 224, 224])

Create Patch-level Data Loaders

In [61]:
dataloader_train = create_data_loader(resized_tensor_train, field_numbers_train, batch_size=config.batch_size, shuffle=True)
dataloader_eval = create_data_loader(resized_tensor_eval, field_numbers_eval, batch_size=config.batch_size, shuffle=False)

### Extract Features

In [81]:
device = 'cuda'
time_steps = 7
in_channels = 10
sentinel2_resnet_extractor = SpectralSentinel2FeatureExtractor()

tf_features_train, train_coord_dl = extract_features(sentinel2_resnet_extractor, dataloader_train, device)
tf_features_eval, eval_coord_dl = extract_features(sentinel2_resnet_extractor, dataloader_eval, device)

print("Resnet 18 (trained on Sentinel-2 images) Extracted Features Shape:", tf_features_train.shape[1])  

ValueError: Unrecognized model in openclimatefix/cloudcasting_example_models. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, audio-spectrogram-transformer, autoformer, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, blenderbot, blenderbot-small, blip, blip-2, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, conditional_detr, convbert, convnext, convnextv2, cpmant, ctrl, cvt, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deformable_detr, deit, depth_anything, deta, detr, dinat, dinov2, distilbert, donut-swin, dpr, dpt, efficientformer, efficientnet, electra, encodec, encoder-decoder, ernie, ernie_m, esm, falcon, falcon_mamba, fastspeech2_conformer, flaubert, flava, fnet, focalnet, f

### K-means

In [ ]:
kmeans = kmeans_function(tf_features_train.cpu(), n_clusters=2, random_state=12)

train_patch_predictions = kmeans.predict(tf_features_train.reshape(tf_features_train.size(0), -1).numpy().astype(np.float32))
eval_patch_predictions = kmeans.predict(tf_features_eval.reshape(tf_features_eval.size(0), -1).numpy().astype(np.float32))

### Evaluation

Ensure all predictions are not zeros/ones

In [ ]:
np.unique(eval_patch_predictions)

array([0, 1], dtype=int32)

Clustering Accuracy: Convert sub-patch level labels to patch-level labels and compare with ground truth

In [ ]:
disease, acc, precision, recall, f1_score, f2_score = evaluate_clustering_metrics_patch_level(field_numbers_eval, eval_patch_predictions, config.labels_path)
print("Disease cluster:", disease)
print("Accuracy:",acc)
print("Precision:",precision)
print("Recall:",recall)
print("F1-score:",f1_score)
print("F2-score:", f2_score)

Disease cluster: 0
Accuracy: 50.82
Precision: 56.1
Recall: 65.71
F1-score: 60.53
F2-score: 63.54


Save Model

In [ ]:
# with open(config.vit_imagenet_path, 'wb') as file:
#     pickle.dump(kmeans, file)

### K-means + Evaluation

In [ ]:
kmeans = kmeans_function(tf_features_train.cpu(), n_clusters=2, random_state=12)

train_patch_predictions = kmeans.predict(tf_features_train.reshape(tf_features_train.size(0), -1).numpy().astype(np.float32))
test_patch_predictions = kmeans.predict(tf_features_test.reshape(tf_features_test.size(0), -1).numpy().astype(np.float32))
eval_patch_predictions = kmeans.predict(tf_features_eval.reshape(tf_features_eval.size(0), -1).numpy().astype(np.float32))

In [ ]:
acc, precision_per_class, recall_per_class, f1_per_class = get_clustering_accuracy(field_numbers_eval, eval_patch_predictions, config.labels_path)
print("Accuracy:",acc)
print("Precision:",precision_per_class)
print("Recall:",recall_per_class)
print("F1-score:",f1_per_class)